In [ ]:
import os
from mido import MidiFile, MidiTrack, Message

In [ ]:
def normalize_duration(ticks, ticks_per_beat):
    """
    Redondea una duración (en ticks) a la subdivisión binaria más cercana.
    """
    # Subdivisiones comunes: redonda, blanca, negra, corchea, etc.
    powers_of_two = [1, 2, 4, 8, 16, 32, 64]
    durations = [int(ticks_per_beat / d) for d in powers_of_two]
    closest = min(durations, key=lambda d: abs(d - ticks))
    return closest

In [ ]:
def normalize_midi_file(input_path):
    """
    Lee y normaliza un archivo MIDI, sobrescribiéndolo con la versión corregida.
    """
    mid = MidiFile(input_path)
    normalized = MidiFile()
    normalized.ticks_per_beat = mid.ticks_per_beat

    for track in mid.tracks:
        new_track = MidiTrack()
        time_buffer = 0
        note_stack = {}

        for msg in track:
            if msg.is_meta:
                new_track.append(msg)
                continue

            time_buffer += msg.time

            if msg.type == 'note_on' and msg.velocity > 0:
                note_stack[msg.note] = time_buffer
                time_buffer = 0

            elif msg.type in ['note_off', 'note_on'] and (msg.velocity == 0 or msg.type == 'note_off'):
                if msg.note in note_stack:
                    start_time = note_stack.pop(msg.note)
                    duration = time_buffer
                    normalized_duration = normalize_duration(duration, mid.ticks_per_beat)

                    new_track.append(Message('note_on', note=msg.note, velocity=64, time=start_time))
                    new_track.append(Message('note_off', note=msg.note, velocity=64, time=normalized_duration))
                    time_buffer = 0

        normalized.tracks.append(new_track)

    normalized.save(input_path)
    print(f"Normalizado y sobrescrito: {input_path}")

In [ ]:
root_folder = r"F:\TG MINTA\separated_songs"

In [ ]:
for dirpath, dirnames, filenames in os.walk(root_folder):
    for filename in filenames:
        if filename.lower().endswith(".mid"):
            midi_path = os.path.join(dirpath, filename)
            try:
                normalize_midi_file(midi_path)
            except Exception as e:
                print(f"Error al procesar {midi_path}: {e}")